In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame, concat
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [49]:
northwestern = pd.read_csv('original_data.csv', sep=";")

# drop all "unnecessary" columns
northwestern = northwestern[northwestern.columns.drop(list(northwestern.filter(regex='facit')))]
northwestern = northwestern[northwestern.columns.drop(list(northwestern.filter(regex='randx')))]
northwestern = northwestern[northwestern.columns.drop(list(northwestern.filter(regex='hadsx')))]
northwestern = northwestern[northwestern.columns.drop(list(northwestern.filter(regex='crqsasx')))]
northwestern = northwestern[northwestern.columns.drop(list(northwestern.filter(regex='eth')))]
# keep a2sexan column
a2sexan = northwestern['a2_cexan']
northwestern = northwestern[northwestern.columns.drop(list(northwestern.filter(regex='a2')))]
northwestern['a2_cexan'] = a2sexan


other_unnecessary_variables = ["CaseID", "copdmonth", "copdday", "copdyear",  "STARTTIME", "ENDTIME", "clinic10ever1_t", "othermeds1_t"]
northwestern = northwestern.drop(other_unnecessary_variables, axis=1)
# drop all columns with more than 90% missing values. Print the dropped columns

# keep only the rows where ds1 contains the word COPD
# northwestern = northwestern[northwestern['ds1'].str.contains("COPD")]
# drop ds1
northwestern = northwestern.drop(["ds1"], axis=1)

# get the feet from the height column
northwestern['feet'] = northwestern['height'].str.split('\'', expand=True)[0]
# get the inches from the height column
northwestern['inches'] = northwestern['height'].str.split('\'', expand=True)[1]
# if inches is '' then set it to 0
northwestern['inches'] = northwestern['inches'].replace('', 0)

# convert feet and inches to cm
northwestern['height'] = (northwestern['feet'].astype(float)*30.48 + northwestern['inches'].astype(float)*2.54).round(2)
# drop feet and inches
northwestern = northwestern.drop(["feet", "inches"], axis=1)

# remove the string characters from weight
northwestern['weight'] = northwestern['weight'].str.replace(" lbs", '')
# convert weight to kg
northwestern['weight'] = (northwestern['weight'].astype(float)*0.45359237).round(2)

# convert categorical values in these columns to numbers: mrexan, smrexan, assyn, mrcx1, birthyr, gender	marstat, educat, occst, income using cat.codes
northwestern['mrexan'] = northwestern['mrexan'].astype('category').cat.codes
northwestern['smrexan'] = northwestern['smrexan'].astype('category').cat.codes
northwestern['assyn'] = northwestern['assyn'].astype('category').cat.codes
northwestern['mrcx1'] = northwestern['mrcx1'].astype('category').cat.codes
northwestern['birthyr'] = northwestern['birthyr'].astype('category').cat.codes
northwestern['gender'] = northwestern['gender'].astype('category').cat.codes
northwestern['marstat'] = northwestern['marstat'].astype('category').cat.codes
northwestern['educat'] = northwestern['educat'].astype('category').cat.codes
northwestern['occst'] = northwestern['occst'].astype('category').cat.codes
northwestern['income'] = northwestern['income'].astype('category').cat.codes



print("Dropped columns with more than 90% missing values: ")
print(set(northwestern.columns).symmetric_difference(set(northwestern.dropna(thresh=0.9*len(northwestern), axis=1).columns)))
northwestern = northwestern.dropna(thresh=0.9*len(northwestern), axis=1)

# turn No into 0 and Yes into 1
northwestern = northwestern.replace(to_replace="No", value=0)
northwestern = northwestern.replace(to_replace="Yes", value=1)

# impute all missing values using mode
northwestern = northwestern.fillna(northwestern.mode().iloc[0])

# export
northwestern.to_csv('northwestern_preprocessed.csv', index=False, sep=";")


Dropped columns with more than 90% missing values: 
{'living', 'ysked', 'clinic10now1_t', 'quita', 'a2_cexan', 'ybeg', 'sked', 'yske', 'psked', 'cigar', 'sterdinhal', 'wquit', 'frequ', 'quitd'}


# Classification

In [50]:

# train a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from yellowbrick.model_selection import FeatureImportances
from lazypredict.Supervised import LazyClassifier

# drop the target variable from the train set
target = northwestern['a2_cexan']

# create a train and test set
train = northwestern.drop(['a2_cexan'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models


# train the model
# rf = RandomForestClassifier(n_estimators=10, random_state=42)
# rf.fit(X_train, y_train)

# predict
# y_pred = rf.predict(X_test)

# evaluate
# print("Accuracy: ", accuracy_score(y_test, y_pred))
# print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))
# print("Classification report: \n", classification_report(y_test, y_pred))

# # feature importance
# viz = FeatureImportances(rf)
# viz.fit(X_train, y_train)
# viz.show()

KeyError: 'a2_cexan'